In [8]:
#0
#datapreproc - moreallfiles
#AAI-540 Group 3 FP

#### Notebook 10
This notebook registers the SimpleBench external benchmark dataset in Amazon Athena for SQL-based analysis.  
The dataset is stored in Amazon S3 and exposed as an external table without duplicating data.  
This enables querying benchmark evaluation results across models and tasks.  
The resulting table supports downstream model comparison and routing performance analysis.
This dataset provides benchmark evaluation results that complement performance and pricing data from other sources.  
Registering it in Athena enables unified querying and integration across all model evaluation datasets.

In [9]:
import pandas as pd
import numpy as np 
import sagemaker

Unable to load JumpStart region config.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/sagemaker/jumpstart/constants.py", line 69, in _load_region_config
    with open(filepath) as f:
         ^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/sagemaker/jumpstart/region_config.json'


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [10]:
simplebench = pd.read_csv('simplebench_external.csv')

In [11]:
simplebench.head(5)

,Model version,Score (AVG@5),Release date,Organization,Country,Training compute (FLOP),Training compute notes,Source,Source link (site from table),Notes,id
0,gemini-3-pro-preview,0.764,2025-11-18,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,Almost certainly >1e25,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recyUT7yjrhcHvcWI
1,gemini-2.5-pro-preview-06-05,0.624,2025-06-05,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recViPEOMOrSB6VKK
2,claude-opus-4-5-20251101,0.620,2025-11-24,Anthropic,United States of America,NaN,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,NaN,NaN,reclKBQ393HPkOMed
3,gpt-5-pro-2025-10-06_high,0.616,2025-10-07,OpenAI,United States of America,NaN,NaN,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recbQ9dn731JxU1au
4,gemini-3-flash-preview,0.611,2025-12-17,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,NaN,SimpleBench Leaderboard,NaN,NaN,rectQAsXBWhS8ne4o


In [12]:
simplebench.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Model version                  52 non-null     object 
 1   Score (AVG@5)                  55 non-null     float64
 2   Release date                   51 non-null     object 
 3   Organization                   52 non-null     object 
 4   Country                        52 non-null     object 
 5   Training compute (FLOP)        21 non-null     float64
 6   Training compute notes         38 non-null     object 
 7   Source                         55 non-null     object 
 8   Source link (site from table)  50 non-null     object 
 9   Notes                          5 non-null      object 
 10  id                             55 non-null     object 
dtypes: float64(2), object(9)
memory usage: 4.9+ KB


In [13]:
original_file = 'simplebench_external.csv'
simplebench.to_csv(original_file, index=False)

In [14]:
sess = sagemaker.Session()
bucket = sess.default_bucket()

!aws s3 cp simplebench_external.csv s3://{bucket}/simplebench_external.csv

upload: ./simplebench_external.csv to s3://sagemaker-us-east-1-907086662522/simplebench_external.csv


In [15]:
#10
#athena db config - simplebench
#AAI-540 Group 3 FP

In [16]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
ingest_create_athena_db_passed = False

In [17]:
database_name = "dsoaws"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [18]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS dsoaws


/tmp/ipykernel_16517/2532319348.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [19]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_16517/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws
2,sagemaker_featurestore


In [20]:
s3_data_path = f"s3://{bucket}/simplebench_external.csv"
s3_table_path = f"s3://{bucket}/table7/"

In [21]:
!aws s3 cp {s3_data_path} {s3_table_path}

copy: s3://sagemaker-us-east-1-907086662522/simplebench_external.csv to s3://sagemaker-us-east-1-907086662522/table7/simplebench_external.csv


In [22]:
#table7 - simplebench.csv
table_name_csv = 'simplebenchmark'
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
ingest_create_athena_table_csv_passed = False
dataexplore = pd.read_csv('simplebench_external.csv')
dataexplore.head()

,Model version,Score (AVG@5),Release date,Organization,Country,Training compute (FLOP),Training compute notes,Source,Source link (site from table),Notes,id
0,gemini-3-pro-preview,0.764,2025-11-18,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,Almost certainly >1e25,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recyUT7yjrhcHvcWI
1,gemini-2.5-pro-preview-06-05,0.624,2025-06-05,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recViPEOMOrSB6VKK
2,claude-opus-4-5-20251101,0.620,2025-11-24,Anthropic,United States of America,NaN,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,NaN,NaN,reclKBQ393HPkOMed
3,gpt-5-pro-2025-10-06_high,0.616,2025-10-07,OpenAI,United States of America,NaN,NaN,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recbQ9dn731JxU1au
4,gemini-3-flash-preview,0.611,2025-12-17,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,NaN,SimpleBench Leaderboard,NaN,NaN,rectQAsXBWhS8ne4o


In [23]:
dataexplore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Model version                  52 non-null     object 
 1   Score (AVG@5)                  55 non-null     float64
 2   Release date                   51 non-null     object 
 3   Organization                   52 non-null     object 
 4   Country                        52 non-null     object 
 5   Training compute (FLOP)        21 non-null     float64
 6   Training compute notes         38 non-null     object 
 7   Source                         55 non-null     object 
 8   Source link (site from table)  50 non-null     object 
 9   Notes                          5 non-null      object 
 10  id                             55 non-null     object 
dtypes: float64(2), object(9)
memory usage: 4.9+ KB


In [24]:
!aws s3 cp {s3_data_path} {s3_table_path}

copy: s3://sagemaker-us-east-1-907086662522/simplebench_external.csv to s3://sagemaker-us-east-1-907086662522/table7/simplebench_external.csv


In [25]:
!aws s3 ls {s3_table_path}

2026-02-23 01:16:51      18481 simplebench_external.csv


### Create Athena External Table

Define the table schema and register the dataset with Athena using an external table pointing to the S3 table directory.

In [26]:
#sql table creation
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         modelversion STRING,
         score FLOAT,
         release_date STRING,
         organization STRING,
         country STRING,
         trainingcomputer FLOAT,
         trainingnotes STRING,
         source STRING,
         sourcelink STRING,
         notes STRING,
         modelid STRING
) ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\\n' 
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_table_path
)

In [27]:
pd.read_sql(statement, conn)

/tmp/ipykernel_16517/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [28]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(8)

/tmp/ipykernel_16517/3158068780.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,aimodelpoll
1,amazon_reviews_parquet
2,amazon_reviews_tsv
3,lifearchitect
4,llmachievements
5,llmleader
6,llmpricing
7,overviewmodel


In [29]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)
df = pd.read_sql(statement, conn)
df.head()

SELECT * FROM dsoaws.simplebenchmark LIMIT 10


/tmp/ipykernel_16517/888292236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,modelversion,score,release_date,organization,country,trainingcomputer,trainingnotes,source,sourcelink,notes,modelid
0,gemini-3-pro-preview,0.764,2025-11-18,Google DeepMind,"""United Kingdom of Great Britain and Northern ...",NaN,,Almost certainly >1e25,SimpleBench Leaderboard,https://simple-bench.com/,
1,gemini-2.5-pro-preview-06-05,0.624,2025-06-05,Google DeepMind,"""United Kingdom of Great Britain and Northern ...",NaN,,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,https://simple-bench.com/,
2,claude-opus-4-5-20251101,0.620,2025-11-24,Anthropic,United States of America,NaN,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,,,reclKBQ393HPkOMed
3,gpt-5-pro-2025-10-06_high,0.616,2025-10-07,OpenAI,United States of America,NaN,,SimpleBench Leaderboard,https://simple-bench.com/,,recbQ9dn731JxU1au
4,gemini-3-flash-preview,0.611,2025-12-17,Google DeepMind,"""United Kingdom of Great Britain and Northern ...",NaN,,,SimpleBench Leaderboard,,


### Summary

This notebook created an Athena external table over the SimpleBench benchmark dataset stored in Amazon S3.  
The table provides SQL access to external benchmark results used to evaluate model performance across tasks.  
This supports downstream model profiling and cost-aware routing workflows.